# Prepare for release your package

## Before release

For a common python package, your code tree will look like:

```
.
└── py_pkg
    ├── LICENSE
    ├── README.md
    ├── py_pkg
    │   ├── __init__.py
    │   └── something.py
    ├── pyproject.toml
    ├── setup.cfg
    └── tests
```

For each file, you should do the following things.

In `py_pkg/__init__.py`, you are supposed to write:

In [1]:
# write your package name here
name = 'py_pkg'

In previous notebook, we have already introuced `pyproject.toml`, a new standard for building python package. Before that, you need two different files to save your package information: `setup.py` for dynamic metadata and `setup.cfg` for static metadata. 

An example for `setup.cfg` file:
```
[metadata]
name = py_pkg
version = 0.0.1
author = xx
author_email = xx@xx.come
description = A small example package
long_description = file: README.md
long_description_content_type = text/markdown
url = https://github.com/xx/xx
classifiers =
    Programming Language :: Python :: 3
    License :: OSI Approved :: MIT License
    Operating System :: OS Independent

[options]
packages = find:
python_requires = >=3.6
```

for `setup.py` file, you can write the similar parameters in `setup.cfg`.

```python
import setuptools

with open("README.md", "r", encoding="utf-8") as fh:
    long_description = fh.read()

setuptools.setup(
    name="py-pkg",
    version="0.0.1",
    author="xx",
    author_email="xx@xx.com",
    description="A small example package",
    long_description=long_description,
    long_description_content_type="text/markdown",
    url="xx",
    classifiers=[
        "Programming Language :: Python :: 3",
        "License :: OSI Approved :: MIT License",
        "Operating System :: OS Independent",
    ],
    packages=setuptools.find_packages(),
    python_requires='>=3.8',
)

```

## Manually package and release

For package upload to pypi, you must add `README.md` and `LICENSE` into your code.

### Generate dist package

In [4]:
! pip -qqq install --upgrade build

In [5]:
! cd .. && python -m build

* Creating virtualenv isolated environment...
[09/08/23 02:25:43] ERROR    pyproject.toml does not contain a ]8;id=914157;file:///home/wangxi/mambaforge-pypy3/envs/torch/lib/python3.10/site-packages/setuptools_scm/_integration/setuptools.py\setuptools.py]8;;\:]8;id=324795;file:///home/wangxi/mambaforge-pypy3/envs/torch/lib/python3.10/site-packages/setuptools_scm/_integration/setuptools.py#119\119]8;;\
                             tool.setuptools_scm section                        
                             Traceback (most recent call                        
                             last):                                             
                               File                                             
                             "/home/wangxi/mambaforge-pypy3/en                  
                             vs/torch/lib/python3.10/site-pack                  
                             ages/setuptools_scm/_integration/                  
                    

Now, you can find dist package in your `dist` dir.

In [6]:
! cd .. && tree dist

dist
├── integration-develop-0.0.0.tar.gz
└── integration_develop-0.0.0-py3-none-any.whl

1 directory, 2 files


### Upload dist package

If you are not familiar with PyPI, you are suggsted to upload dist to `Test PyPI` firstly, which design for green hand to test. After registration, you can use `twine` to upload your package to test pypi.

In [9]:
! pip -qqq install --user --upgrade twine

使用下面的命令上传dist分发包到`PyPI`：

```bash
python -m twine upload --repository testpypi dist/*
```

After you test your package via `pip install --index-url https://test.pypi.org/simple/ --no-deps py-pkg`, you can upload your package to PyPI now.

```bash
python -m twine upload dist/*
```

## Automatic upload package (Recommended)

For integrated and easy-control develop, you are recommended to use CI/CD to automatically upload your distributed packages into PyPI.

Here is an example of github action yaml file:

```yaml
  release:
    if: startsWith(github.event.ref, 'refs/tags/v')
    needs: [lint, test]
    environment: pypi
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      - uses: actions/setup-python@v4
        with:
          python-version: 3.x
          cache: "pip"
      - name: Install dependencies for building
        run: pip install wheel setuptools_scm
      - name: build package
        run: python setup.py sdist bdist_wheel
      - name: create release
        uses: "marvinpinto/action-automatic-releases@latest"
        with:
          repo_token: "${{ secrets.GITHUB_TOKEN }}"
          prerelease: false
          files: |
            dist/*
      - name: publish to PyPI
        uses: pypa/gh-action-pypi-publish@release/v1
        with:
          password: ${{ secrets.PYPI_API_TOKEN }}
```